<a href="https://colab.research.google.com/github/KSG722/class2022Spring/blob/main/huggingface_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
import gradio as gr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.1 MB 4.3 MB/s 
     |████████████████████████████████| 256 kB 45.4 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 212 kB 53.2 MB/s 
     |████████████████████████████████| 1.1 MB 27.5 MB/s 
     |████████████████████████████████| 2.0 MB 17.4 MB/s 
     |████████████████████████████████| 84 kB 1.7 MB/s 
     |████████████████████████████████| 54 kB 651 kB/s 
     |████████████████████████████████| 271 kB 45.4 MB/s 
     |████████████████████████████████| 144 kB 35.5 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 11.1 MB 38.2 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 80 kB 8.2 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 4.0 M

### [Image classification](https://huggingface.co/tasks/image-classification)

직접 링크로 타고 들어가서 설명 읽어보기(시험문제는 이 걸 할줄 안다는 전제로 나올 수도)
이미지를 넣으면 각각의 확률값과 함게 class(ex. 이집트 고양이)가 분류된다.

e.g. https://huggingface.co/google/vit-base-patch16-224 \
How to use

구체적인 image classification 모델

how to use에 적힌 코딩을 긁어오자

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 4.7 MB/s 
     |████████████████████████████████| 6.6 MB 58.4 MB/s 
     |████████████████████████████████| 84 kB 2.8 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Predicted class: Egyptian cat


url은 딱봐도 그림 이미지. pertrained는 행렬. 즉 url 이미지라는 input에 행렬값들을 곱해서 output으로 class를 분류해 도출해내는 것.
-> 이집션 캣인지 호랑이인지 강아지인지 확률값으로 계산 한 후 가장 확률이 높은 것으로 classify. 이집션 캣이 0.98xx값이 나옴. 참고로 값을 다 더하면 1.


Demo in Gradio

In [ ]:
def func (image):
  feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
  model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

  inputs = feature_extractor(images=image, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  # model predicts one of the 1000 ImageNet classes
  predicted_class_idx = logits.argmax(-1).item()
  predicted_class = model.config.id2label[predicted_class_idx]
  return predicted_class

In [ ]:
import os
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/tiger.jpg"
os.system("curl " + url + " > tiger.jpg")
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/dog.jpg"
os.system("curl " + url + " > dog.jpg")

0

두개의 url은 class 사이트에 있는 이미지. 호랑이와 강아지 그림. 이 url들을 받아서 os.system 함수에 적어주면 >이하의 이름으로 url 이미지 파일을 다운로드(좌측 상단).

In [ ]:
gr.Interface(fn=func, inputs='image', outputs='text', examples = ['tiger.jpg', 'dog.jpg']).launch()

그라디오화. 입력의 형태와 출력의 형태를 정의해줘야함. 입력은 이미지를 입력하고, 출력은 텍스트로 뱉길 원함. 위에 있는 함수의 입력 유형과 출력유형을 그대로 적어주는 것.


examples는 아까 다운로드받은 두개의 tiger, dog 이미지 파일을 그라디오에서 예시로 띄워주는 것. 이렇게 띄워주면 굳이 drag&drop 방식으로 이미지를 가져올 필요 없이 example를 클릭하기만 하면 됨.

### [Fill-Mask](https://huggingface.co/tasks/fill-mask)

빈칸 채우기

e.g. https://huggingface.co/bert-base-uncased \
How to use

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Hello I'm a [MASK] model.")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

[{'score': 0.10731077939271927,
  'sequence': "hello i'm a fashion model.",
  'token': 4827,
  'token_str': 'fashion'},
 {'score': 0.0877445638179779,
  'sequence': "hello i'm a role model.",
  'token': 2535,
  'token_str': 'role'},
 {'score': 0.05338413640856743,
  'sequence': "hello i'm a new model.",
  'token': 2047,
  'token_str': 'new'},
 {'score': 0.046672213822603226,
  'sequence': "hello i'm a super model.",
  'token': 3565,
  'token_str': 'super'},
 {'score': 0.027095887809991837,
  'sequence': "hello i'm a fine model.",
  'token': 2986,
  'token_str': 'fine'}]

Hello I'm a [MASK] model 이 문장을 가지고 실험하는 것

여기서 결과물은 dictionary 여러개가 list로 묶여있음. 즉 list라는 것. (가장 밖의 기호가 [ ] 이므로)

Demo in Gradio

In [ ]:
import pandas as pd
def func (text):
  unmasker = pipeline('fill-mask', model='bert-base-uncased')
  result = unmasker(text)
  df = pd.DataFrame(result)
  return df

먼저 인풋 아웃풋을 정의하는 fuction을 정의. 모델을 불러와서 unmasker라고 했고, 그 언메스커로부터 text를 받아와서 결과물이라고 정의.

아까 위 함수의 결과값이 list였으므로.. 데이터 프레임으로 전환해주자.

In [ ]:
examples = ["Hello I'm a [MASK] model.", "It is raining outside. I feel [MASK]."]

역시나 예시를 띄어주기 위해 미리 정의

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://23552.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7fcb69179d90>,
 'http://127.0.0.1:7860/',
 'https://23552.gradio.app')

그라디오 실현.

### [Token classification](https://huggingface.co/tasks/token-classification)

natural language understading. 어떤 자연어가 주어져 있다면(문장이나 텍스트), 엄청 자유도가 높을 것. 즉 아무 문장이나 단어나 다 가능. 그러나 컴퓨터는 자유도가 높은 환경에 적합하지 않음. 따라서 컴퓨터는 여기서 필요한 단어나 정보만을 추출. Named Entity Recognition이나 Part of Speech가 그 중 하나의 기술. 특정 명사를 AI가 캐치해내는 것. 로케이션 정보(LOC)와 조직/단체 정보(ORG), 혹은 사람에 대한 정보(PER)를 위주로 캐치해냄. 

예를 들어 my name is Wolfgang and I live in Berlin이라는 문장이 있으면 다른 건 다 버리고 Wolfgang은 PER로 인식되고 Berlin은 LOC로 추출된다. 

e.g. https://huggingface.co/dslim/bert-base-NER \
How to use

In [ ]:
!pip install transformers

recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

자연어에는 많은 정보가 있지만 그 중에서도 위의 4개의 정보가 가장 중요하다.

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


tokenizer는 stirng의 형태로 문장을 쪼개주는 것. 단어 수준의 형태로 문장을 쪼갬. 본격적인 처리에 앞서서 전처리라고 할 수 있음.
model은 각각의 string을 분류하는 함수.

nlp는 이가 종합된 모델. example로 정의된 문장을 이 모델에 넣고 돌리면 각각의 토큰에 대한 설명이 결과로 도출됨.

Demo in Gradio

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
def func (text):
  tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
  model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
  nlp = pipeline("ner", model=model, tokenizer=tokenizer)
  result = nlp(text)
  df = pd.DataFrame(result)
  return df

In [ ]:
examples = ["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

그라디오 돌리면 Berlin과 Seoul이 Loc으로 나오고 Chris와 Wolfgang은 PER로 잘 인식됨. 스코어가 확률값.

### [Sentence similarity](https://huggingface.co/tasks/sentence-similarity)

raw data를 feature 형태로 한번 컨버팅해줌. 그러면 vector(embedding)로 바뀌어 나오는데 이렇게 바꾸면 두 텍스트들 간에 유사성을 쉽게 계산. 

e.g. https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 \
How to use

In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 3.8 MB/s 
     |████████████████████████████████| 1.2 MB 11.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=d5d0fd2a5ff0b3aeb5f911f5489eddbc786325ebc5dc0514b31248c69aab2c14
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "it is one example writing"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)

Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

[[ 1.88456848e-01  1.74256176e-01  5.44777215e-02  2.90517420e-01
   1.67664081e-01 -4.72069457e-02  6.45580173e-01  1.59808800e-01
   2.26892605e-01 -3.08905598e-02  2.55883545e-01 -5.25876768e-02
  -2.26101175e-01 -5.71063384e-02  1.30426243e-01  1.24953501e-01
   3.17496210e-01  1.94444105e-01 -5.86325705e-01 -1.25861000e-02
   6.09909177e-01  1.64327413e-01  3.33113708e-02 -2.73830682e-01
  -2.89757580e-01 -2.11196810e-01 -2.26138867e-02 -1.70359373e-01
   1.61589950e-01  6.08273745e-02 -2.41624042e-01  1.85792103e-01
   4.27409589e-01  1.92951798e-01 -7.23447800e-02  1.66111022e-01
   1.04428008e-01  2.04772323e-01  2.11167201e-01  1.99740008e-01
  -9.40827355e-02 -1.73836619e-01  6.42733201e-02  2.80254930e-01
  -2.95305759e-01  6.20952919e-02  1.04276791e-01 -2.36439407e-02
   1.29131526e-01 -1.26174763e-01 -1.78990051e-01  3.70057598e-02
  -6.12506211e-01  5.02983071e-02  1.77303478e-01  2.24941239e-01
   1.73860714e-01 -3.84028591e-02 -2.12868378e-01  2.58492470e-01
  -1.21016

In [ ]:
len(embeddings)

2

임베딩에는 2개의 리스트가 있음

In [ ]:
len(embeddings[0])

384

그리고 첫번째 리스트에는 384개의 숫자가 있음.

embeddings[1]에도 384개의 숫자있음.

In [ ]:
cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
cosine_scores

tensor([[0.5726]])

무슨 코사인 각도 어쩌고로 유사성 비교

Demo in Gradio

In [ ]:
def func (text1, text2):
  from sentence_transformers import SentenceTransformer, util
  model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
  embeddings = model.encode(text1, text2)
  cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
  return cosine_scores

코사인 어쩌고를 다 합쳐서 fuction화한 것.

In [ ]:
examples = ["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]

In [ ]:
gr.Interface(fn=func, inputs=['text', 'text'], outputs='number', examples = examples).launch()

ValueError: ignored

인풋에서 text를 2개 설정해줬떠니 오류 뜨네 왜지..?